# Shamir Secret Sharing (SS)

This is demo for our implementation of Shamir's secret sharing. We choose the Field $\mathbb{Z}_p$ where $p > 2^\lambda$ is a prime ($\lambda$ is the key size)

In [1]:
from ftsa.protocols.buildingblocks.ShamirSS import SSS
import random

threshold = 70
nclients  = 100
SS = SSS(1024)

number1 = random.randint(0,2**1024-1)
number2 = random.randint(0,2**1024-1)

allshares1 = SS.share(threshold, nclients, number1)
allshares2 = SS.share(threshold, nclients, number2)

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

lagcoef = SS.lagrange(shares1)
reconstucted1 =  SS.recon(shares1, lagcoef)
reconstucted2 =  SS.recon(shares2, lagcoef)
reconstucted =  SS.recon(shares, lagcoef)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Shamir Secret Sharing ove the Integers (ISS)

This is demo for our implementation of Shamir's secret sharing over the integers. We choose the security parameter sigma to $\sigma = 128$ bits

In [2]:
from ftsa.protocols.buildingblocks.IntegerSS import ISSS
import random
from math import factorial

sigma = 128
threshold = 70
nclients  = 100
delta = factorial(nclients)

SS = ISSS(1024, sigma)

number1 = random.randint(-2**1024+1,2**1024-1)
number2 = random.randint(-2**1024+1,2**1024-1)

allshares1 = SS.Share(number1,threshold, range(1,nclients+1))
allshares2 = SS.Share(number2,threshold, range(1,nclients+1))

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

reconstucted1 =  SS.Recon(shares1, threshold, delta)
reconstucted2 =  SS.Recon(shares2, threshold, delta)
reconstucted =  SS.Recon(shares, threshold, delta)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Deffie-Hellman Key Aggreement (KA)
It uses ECDH p256 curve. It then computes with SHA256 over the ECDH agreed key to generate a secret key for AES (256 bit)

In [3]:
from ftsa.protocols.buildingblocks.KeyAggreement import KAS

sharedkeysize = 1024

KA1 = KAS()
KA2 = KAS()

KA1.generate()
KA2.generate()

# check if the shared secrets match
print("Verify:", KA1.agree(KA2.pk, sharedkeysize) == KA2.agree(KA1.pk, sharedkeysize))

# print shared key
sharedkey = KA1.agree(KA2.pk, sharedkeysize)
print("key: " + str(sharedkey))

Verify: True
key: 113354440650051606814986025038707598068744197150103086302284261259742951141923712450817345067341764792955694275939892613685339210332942026762227636446330092904684249063116914268562818346600154785016360642318803008622437026497807955478696318525049957635691331180898877973650172230778860365136148871773379344159


In [4]:
# test serializing secret keys
sk1_bytes = KA1.get_sk_bytes()
KA = KAS()
KA.generate_from_bytes(sk1_bytes)
print("Verify:", KA.sk == KA1.sk and  KA.pk == KA1.pk)

Verify: True


# AES-GCM Authenticated Encryption (AE)

In [5]:
import ftsa.protocols.buildingblocks.AESGCM128 as aes

message=b"TOP SECRET"

key = aes.EncryptionKey(sharedkey)
e = key.encrypt(message)
m = key.decrypt(e)
print("Verify:", message == m)

Verify: True


# Vector Encoding Scheme (VE)
This is a vector to vector encoding. It batches several elements of the vectors into a single integer. The batch size depends on the size of that integer (i.e. this means the plaintext size of an encryption scheme). It also depends on the number of safety bits need to be preserved for each vector element (this is to support addition operation without reaching an overflow)

In [6]:
from ftsa.protocols.buildingblocks.VectorEncoding import VES
import random 

keysize = 2048
nclients = 100
valuesize = 32
vectorsize = 1000

VE = VES(keysize, nclients, valuesize, vectorsize)
vector = [random.randint(0,2**32-1) for _ in range(vectorsize)]

# encode the vector 
E = VE.encode(V=vector)
print("encoded decreased the vector size from {} to {}".format(len(vector),len(E)))

# decode the vector 
V = VE.decode(E=E)
print("Verify:", V == vector)

encoded decreased the vector size from 1000 to 20
Verify: True


# Pseudo Randomg Generator (PRG)
## We use AES-CTR to generate a pseudo random vector from a seed

The demo first generates a random seed. The seed is used as the AES key to encrypt a '0x00'-repeated message of large size. The encryption result is transformed into a vector of integers. The output vector size is controlled by the size of the input message.

In [7]:
from ftsa.protocols.buildingblocks.PRG import PRG
import random

securitybits = 128 #bits
elementsize = 16 #bits
vectorsize = 2**20 #elements

# setup the PRG 
prg = PRG(vectorsize, elementsize)

In [8]:
# get a random seed of 16 bytes long
b = random.SystemRandom().getrandbits(securitybits).to_bytes(securitybits // 8,"big")

# generate a random vector from that seed 
B = prg.eval(b)

print("Generated a pseudo random vector of length {} ({} bits elements) from {} bits random value".format(len(B), elementsize, securitybits))
B

Generated a pseudo random vector of length 1048576 (16 bits elements) from 128 bits random value


[21819,
 18496,
 41033,
 48814,
 64952,
 5382,
 6882,
 49000,
 41468,
 5194,
 14518,
 52299,
 57331,
 41620,
 36577,
 44337,
 26008,
 30175,
 31876,
 37844,
 13502,
 59623,
 43324,
 40196,
 34247,
 37503,
 60930,
 38268,
 57740,
 39366,
 40792,
 55017,
 17116,
 17786,
 3270,
 61727,
 62420,
 41208,
 43622,
 45015,
 60207,
 55001,
 61763,
 50697,
 64412,
 46025,
 7260,
 35659,
 41928,
 41333,
 30914,
 31565,
 26063,
 6365,
 54248,
 16759,
 24077,
 53219,
 28927,
 51277,
 19999,
 27643,
 6237,
 15941,
 45670,
 21949,
 47652,
 7596,
 30806,
 30921,
 21344,
 35851,
 62707,
 9679,
 56237,
 29398,
 37399,
 862,
 59495,
 39030,
 3900,
 43489,
 25793,
 19087,
 21244,
 61888,
 47513,
 41817,
 51199,
 16740,
 34745,
 32195,
 1813,
 62234,
 6927,
 16107,
 55456,
 49589,
 5534,
 819,
 51186,
 46831,
 19876,
 26765,
 35877,
 35294,
 49441,
 57906,
 17491,
 10841,
 8888,
 46359,
 53283,
 33271,
 9437,
 42597,
 20730,
 35091,
 59726,
 50536,
 24326,
 4949,
 30339,
 14049,
 48316,
 52235,
 52706,
 596